# Multiple classifiers

We will attempt to create methods to train and evaluate multiple multi-output classifier and rank them based on F2 score

Given we were not able to create a good balanced dataset, we will test using original dataset with reduced features 

In [30]:
%pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [31]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xb

In [32]:
X_train=pd.read_csv('X_train.csv')
y_train=pd.read_csv('y_train.csv')
X_test=pd.read_csv('X_test.csv')
y_test=pd.read_csv('y_test.csv')

y_train_omf = y_train['Machine failure']
y_test_omf = y_test['Machine failure']

X_train_reduced = X_train.drop(columns=['Torque _Nm_', 'Process temperature _K_', 'Air temperature _K_'])
X_test_reduced = X_test.drop(columns=['Torque _Nm_', 'Process temperature _K_', 'Air temperature _K_'])


In [67]:
from sklearn.metrics import fbeta_score, precision_score, recall_score, accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.multioutput import MultiOutputClassifier

def get_ml_perf_machine_failure(name, clf, X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test):
    y_train_omf = y_train['Machine failure']
    y_test_omf = y_test['Machine failure']
    clf = MultiOutputClassifier(clf)
    clf.fit(X_train, y_train)
    y_train_pred= clf.predict(X_train)
    y_train_pred = pd.DataFrame(y_train_pred, columns=y_test.columns)
    y_test_pred = clf.predict(X_test)
    y_test_pred = pd.DataFrame(y_test_pred, columns=y_test.columns)
    y_test_pred_omf = y_test_pred['Machine failure']
    y_train_pred_omf = y_train_pred['Machine failure']
    f2_train=fbeta_score(y_train_omf, y_train_pred_omf, beta=2)
    f2_test=fbeta_score(y_test_omf, y_test_pred_omf, beta=2)
    train_recall=recall_score(y_train_omf, y_train_pred_omf)
    test_recall=recall_score(y_test_omf, y_test_pred_omf)
    train_precision=precision_score(y_train_omf, y_train_pred_omf, zero_division=0)
    test_precision=precision_score(y_test_omf, y_test_pred_omf, zero_division=0)
    train_accuracy=accuracy_score(y_train_omf, y_train_pred_omf)
    test_accuracy=accuracy_score(y_test_omf, y_test_pred_omf)
    predictor_name = clf.__class__.__name__
    cm = confusion_matrix(y_train_omf, y_train_pred_omf)
    train_tn, train_fp, train_fn, train_tp = cm.ravel()
    cm = confusion_matrix(y_test_omf, y_test_pred_omf)
    test_tn, test_fp, test_fn, test_tp = cm.ravel()
    result = {
        "multi-output-classifier": clf,
        "f2_train": f2_train,
        "f2_test": f2_test,
        "train_accuracy": train_accuracy,
        "test_accuracy": test_accuracy,
        "train_recall":train_recall,
        "test_recall": test_recall,
        "test_precision": test_precision,
        "train_precision":train_precision,
        "predictor_name": predictor_name,
        "train_fp": train_fp,
        "train_fn": train_fn,
        "test_fp": test_fp,
        "test_fn": test_fn,
        "train_tp": train_tp,
        "train_tn": train_tn,
        "test_tp": test_tp,
        "test_tn": test_tn,
        "test_pred": y_test_pred,
        "train_pred": y_train_pred,
        "name": name        
    }
    return result

def evaluate_models(models:dict,  X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test):
    results = []
    for name, model in models.items():
        results.append(get_ml_perf_machine_failure(name, model, X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test))
    results.sort(reverse=True, key=lambda result: result["f2_test"])
    print(" -- Sorted result --")
    for result in results:
        print(f"{result["name"]:20} : f2-test {result["f2_test"]:20} | accuracy {result["test_accuracy"]:10}  | test-fp {result["test_fp"]:10}  | test-fn {result["test_fn"]:10} | f2-train {result["f2_train"]:20} |")
    return results

In [71]:
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

names = [
    "Nearest Neighbors",
    "Decision Tree",
    "Random Forest",
    "AdaBoost",
    "Naive Bayes",
    "XGBoost"
]

classifiers = [
    KNeighborsClassifier(5),
    DecisionTreeClassifier(random_state=42, max_depth=5, min_samples_leaf=10, min_samples_split=30),
    RandomForestClassifier(random_state=42,max_depth=5, min_samples_leaf=10, min_samples_split=30 ),
    AdaBoostClassifier(random_state=42),
    GaussianNB(),
    XGBClassifier(n_estimators=100,objective='binary:logistic',random_state=42, max_depth=5)
]

models = {name: clf for name, clf in zip(names, classifiers)}

results_reduced = evaluate_models(models, X_train=X_train_reduced, X_test=X_test_reduced) # use reduced features

best_clf = results_reduced[0]["multi-output-classifier"]

best_clf


 -- Sorted result --
Decision Tree        : f2-test   0.8504398826979472 | accuracy     0.9925  | test-fp          3  | test-fn         12 | f2-train   0.8040388930441287 |
XGBoost              : f2-test   0.8504398826979472 | accuracy     0.9925  | test-fp          3  | test-fn         12 | f2-train   0.9884559884559885 |
Random Forest        : f2-test   0.6382978723404256 | accuracy     0.9825  | test-fp          7  | test-fn         28 | f2-train   0.6147859922178989 |
AdaBoost             : f2-test   0.4716981132075472 | accuracy      0.976  | test-fp          8  | test-fn         40 | f2-train  0.47351524879614765 |
Naive Bayes          : f2-test  0.41420118343195267 | accuracy      0.964  | test-fp         30  | test-fn         42 | f2-train   0.4241741741741742 |
Nearest Neighbors    : f2-test  0.40192926045016075 | accuracy     0.9745  | test-fp          6  | test-fn         45 | f2-train   0.4971932638331997 |


MultiOutputClassifier(estimator=DecisionTreeClassifier(max_depth=5,
                                                       min_samples_leaf=10,
                                                       min_samples_split=30,
                                                       random_state=42))

### Save Best Classifier/Predictor

In [72]:
import pickle
with open('ml_predictor.pkl', 'wb') as f:
    pickle.dump(best_clf, f)
print("Best ML Predictor model saved to 'ml_predictor.pkl'")

Best ML Predictor model saved to 'ml_predictor.pkl'


## Conclusion

Our Decision Tree classifier is simple and yet perform sufficiently for our need as it matches XGBoost performance